In [22]:
import ee
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
ee.Initialize()

In [24]:
# Define the area of interest
lat = 12.793
lon = -16.770

b = 'data/Senegal.shp'
roi = geemap.shp_to_ee(b)


In [4]:
# Declare the multiple dates. #TODO automated timestep
sd1 = '2016-01-01'
ed1 = '2016-12-31'
sd2 = '2017-01-01'
ed2 = '2017-12-31'
sd3 = '2018-01-01'
ed3 = '2018-12-31'

In [16]:
# The main function, receiving only date range and roi
# Imagecollection filter
# cloudmask
# compositing and reducing
# MNDWI

band_name = ee.List(['B2','B3', 'B4', 'B8','B12'])
def imgcol(sdate, edate, aoi):
    year = ee.String(sdate).slice(0,4)
    #Filter cloud 
    def cloud_prob():
        innerJoined = ee.Join.inner().apply(**{
            'primary':ee.ImageCollection("COPERNICUS/S2")\
                .filterBounds(aoi)\
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)),
             'secondary': ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")\
                .filterBounds(aoi),       
            'condition':ee.Filter.equals(**{
                'leftField':'system:index',
                'rightField':'system:index',
            })
        })
        def mergeBands(joinresult):
            return ee.Image(joinresult.get('primary'))\
                .addBands(joinresult.get('secondary'))
        newCollection = innerJoined.map(mergeBands)
        return ee.ImageCollection(newCollection)

    #cloud mask
    def maskcloud(_image):
        cloudProbabilityThreshold =20
        cloudMask = _image.select('probability').lt(cloudProbabilityThreshold)
        return _image.updateMask(cloudMask)

    #Less clouded image collection with date range
    _imagecoll = cloud_prob().filterDate(sdate,edate)
    
    #cloud mask collection
    __imagecoll = _imagecoll.map(maskcloud).select(band_name)
    
    #resolution merge - B12(20m) to 10m
    def resmerge(img):
        #generate a panchrmatic band from exist 10m bands 
        panchro = (img.select('B2').add(img.select('B3'))\
                       .add(img.select('B4')).add(img.select('B8')))\
                       .divide(4).rename('pan')
        img = img.addBands(panchro)

        bands = ['B12']
        #a kernel function to sharpen the 20m band to 10m
        def sharpen(img):
            kernel = ee.Kernel.square(**{
                'radius': 20,
                'units':'meters'
            })
            toSharpen = img.select(bands)
            pan = img.select('pan')
            meanPan = pan.reduceNeighborhood(**{
                'reducer' : ee.Reducer.mean(),
                'kernel' : kernel
            })
            offset = pan.subtract(meanPan)
            return toSharpen.add(offset)
        sharpened = sharpen(img)
        band10m = img.select(['B2','B3','B4','B8'])
        imagef = band10m.addBands(sharpened)
        return imagef

    image_collection = __imagecoll.map(resmerge)
    #compositing - median most appropriate
    image_feature = image_collection.median().clip(aoi).divide(10000)

    #MNDWI
    mndwi = image_feature.expression(
    '(GREEN - SWIR)/(GREEN+SWIR)',{
        'GREEN':image_feature.select('B3'),
        'SWIR':image_feature.select('B12')
    }).rename(year.cat('_MNDWI'))
    return mndwi

In [17]:
#
mndwi2016 = imgcol(sd1,ed1,roi)
mndwi2017 =imgcol(sd2,ed2,roi)
mndwi2018 =imgcol(sd3,ed3,roi)
# print(mndwi2016.getInfo())

In [18]:
#An image container to hold all the timeseries MNDWI
NMDVI_Image = ee.Image()
NMDVI_Image = NMDVI_Image.addBands(mndwi2016).addBands(mndwi2017).addBands(mndwi2018)
print(NMDVI_Image.bandNames().getInfo())

['constant', '2016_MNDWI', '2017_MNDWI', '2018_MNDWI']


In [14]:
Map = geemap.Map(center=(lat,lon),zoom=8)
Map

Map(center=[12.793, -16.77], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [19]:
vis_params = {'bands': ['2016_MNDWI'], 'palette': ['fff7f3', ' fff6f2', ' fff6f2', ' fff5f1', ' fff4f0', ' fff3f0', ' fff3ef', ' fff2ee', ' fef1ed', ' fef1ed', ' fef0ec', ' feefeb', ' feeeeb', ' feeeea', ' feede9', ' feece9', ' feebe8', ' feebe7', ' feeae7', ' fee9e6', ' fee9e5', ' fee8e5', ' fee7e4', ' fee6e3', ' fde6e2', ' fde5e2', ' fde4e1', ' fde4e0', ' fde3e0', ' fde2df', ' fde1de', ' fde1de', ' fde0dd', ' fddfdc', ' fddedb', ' fdddda', ' fdddd9', ' fddcd8', ' fddbd7', ' fddad7', ' fdd9d6', ' fdd8d5', ' fdd7d4', ' fdd7d3', ' fdd6d2', ' fdd5d1', ' fdd4d0', ' fdd3cf', ' fcd2ce', ' fcd1cd', ' fcd1cd', ' fcd0cc', ' fccfcb', ' fcceca', ' fccdc9', ' fcccc8', ' fcccc7', ' fccbc6', ' fccac5', ' fcc9c4', ' fcc8c3', ' fcc7c3', ' fcc6c2', ' fcc6c1', ' fcc5c0', ' fcc4c0', ' fcc2bf', ' fcc1bf', ' fcc0bf', ' fcbfbe', ' fcbebe', ' fcbcbd', ' fbbbbd', ' fbbabd', ' fbb9bc', ' fbb8bc', ' fbb6bc', ' fbb5bb', ' fbb4bb', ' fbb3bb', ' fbb2ba', ' fbb0ba', ' fbafba', ' fbaeb9', ' fbadb9', ' fbacb9', ' fbaab8', ' fba9b8', ' faa8b8', ' faa7b7', ' faa6b7', ' faa5b7', ' faa3b6', ' faa2b6', ' faa1b6', ' faa0b5', ' fa9eb5', ' fa9db4', ' fa9bb4', ' fa99b3', ' fa97b2', ' f996b2', ' f994b1', ' f992b0', ' f991b0', ' f98faf', ' f98dae', ' f98bae', ' f98aad', ' f988ad', ' f986ac', ' f984ab', ' f883ab', ' f881aa', ' f87fa9', ' f87ea9', ' f87ca8', ' f87aa8', ' f878a7', ' f877a6', ' f875a6', ' f873a5', ' f871a4', ' f770a4', ' f76ea3', ' f76ca3', ' f76ba2', ' f769a1', ' f767a1', ' f666a1', ' f564a0', ' f462a0', ' f361a0', ' f35f9f', ' f25d9f', ' f15c9f', ' f05a9e', ' ef599e', ' ee579e', ' ee559d', ' ed549d', ' ec529d', ' eb509c', ' ea4f9c', ' ea4d9c', ' e94b9c', ' e84a9b', ' e7489b', ' e6479b', ' e5459a', ' e5439a', ' e4429a', ' e34099', ' e23e99', ' e13d99', ' e13b98', ' e03a98', ' df3898', ' de3697', ' dd3597', ' dc3397', ' db3196', ' d93095', ' d82e94', ' d62d93', ' d52b93', ' d32992', ' d22891', ' d02690', ' cf258f', ' cd238f', ' cc218e', ' ca208d', ' c91e8c', ' c71d8c', ' c61b8b', ' c4198a', ' c31889', ' c21688', ' c01588', ' bf1387', ' bd1186', ' bc1085', ' ba0e84', ' b90d84', ' b70b83', ' b60982', ' b40881', ' b30681', ' b10580', ' b0037f', ' ae017e', ' ad017e', ' ab017e', ' aa017d', ' a8017d', ' a6017d', ' a5017d', ' a3017d', ' a1017c', ' a0017c', ' 9e017c', ' 9c017c', ' 9b017b', ' 99017b', ' 98017b', ' 96017b', ' 94017b', ' 93017a', ' 91017a', ' 8f017a', ' 8e017a', ' 8c0179', ' 8b0179', ' 890179', ' 870179', ' 860179', ' 840178', ' 820178', ' 810178', ' 7f0178', ' 7d0177', ' 7c0177', ' 7a0177', ' 790177', ' 770176', ' 760176', ' 740175', ' 730175', ' 710175', ' 6f0174', ' 6e0174', ' 6c0173', ' 6b0173', ' 690173', ' 680172', ' 660172', ' 650171', ' 630171', ' 620171', ' 600070', ' 5f0070', ' 5d006f', ' 5b006f', ' 5a006e', ' 58006e', ' 57006e', ' 55006d', ' 54006d', ' 52006c', ' 51006c', ' 4f006c', ' 4e006b', ' 4c006b', ' 4b006a', ' 49006a'], 'min': -0.5, 'max': 0.5}

In [20]:
Map.addLayer(NMDVI_Image,vis_params, 'MNDWI')
# Map.addLayer(roi,{}, 'Roi')

https://www.reddit.com/r/remotesensing/comments/es3mse/error_in_display_url_image_from_google_earth/ff866gf/
https://gis.stackexchange.com/questions/340340/layer-error-element-get-argument-property-invalid-type-expected-string-a
https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard#static_mapping_of_land_surface_temperature_and_ground_elevation
https://www.sciencedirect.com/science/article/pii/S1364815219300490#fig1
https://sandbox.digitalearth.africa/user/ecue000/lab?